In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(subset=['Trade_History'], inplace=True)
    df['Trade_History'] = df['Trade_History'].apply(ast.literal_eval)
    return df

In [7]:
def identify_trade_position(trade):
    side = trade['side'].lower()
    position_side = trade['positionSide'].lower()
    return f"{position_side}_{'open' if side == 'buy' else 'close'}"

In [8]:
def calculate_metrics(trade_history):
    if not trade_history:
        return {
            'ROI': 0,
            'PnL': 0,
            'Sharpe Ratio': 0,
            'Max Drawdown': 0,
            'Win Rate': 0,
            'Win Positions': 0,
            'Total Positions': 0
        }

    initial_balance = trade_history[0]['quantity']
    total_pnl = sum(trade['realizedProfit'] for trade in trade_history)
    roi = (total_pnl / initial_balance) * 100 if initial_balance else 0

    profits = [trade['realizedProfit'] for trade in trade_history if trade['realizedProfit'] > 0]
    losses = [trade['realizedProfit'] for trade in trade_history if trade['realizedProfit'] < 0]

    win_positions = len(profits)
    total_positions = len(trade_history)
    win_rate = (win_positions / total_positions) * 100 if total_positions else 0

    returns = np.array(profits + losses)
    sharpe_ratio = np.mean(returns) / np.std(returns) if np.std(returns) else 0

    equity_curve = np.cumsum(returns)
    max_drawdown = np.max(np.maximum.accumulate(equity_curve) - equity_curve) if len(equity_curve) > 0 else 0

    return {
        'ROI': roi,
        'PnL': total_pnl,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Win Positions': win_positions,
        'Total Positions': total_positions
    }

In [9]:
# Rank accounts based on metrics
def rank_accounts(df):
    metrics_list = []
    for _, row in df.iterrows():
        metrics = calculate_metrics(row['Trade_History'])
        metrics['Port_IDs'] = row['Port_IDs']
        metrics_list.append(metrics)

    df_metrics = pd.DataFrame(metrics_list)
    df_metrics['Score'] = df_metrics['ROI'] + df_metrics['Sharpe Ratio'] - df_metrics['Max Drawdown'] + df_metrics['Win Rate']
    df_ranked = df_metrics.sort_values(by='Score', ascending=False)

    return df_ranked.head(20)

In [10]:
def main(file_path, save_path):
    df = load_data(file_path)
    top_20_accounts = rank_accounts(df)
    print(top_20_accounts)

    # Save output to Google Drive
    top_20_accounts.to_csv(save_path, index=False)
    print(f"Results saved to: {save_path}")

In [11]:
file_path = "/content/drive/MyDrive/TRADES_CopyTr_90D_ROI.csv"
save_path = "/content/drive/My Drive/top_20_accounts.csv"  # Adjust based on your Google Drive structure
main(file_path, save_path)

/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


              ROI           PnL  Sharpe Ratio  Max Drawdown   Win Rate  \
16   49658.226912   4545.242338      0.428111     40.286009  24.172185   
132  41673.340182   1884.468443      0.127827    974.465799  46.452867   
136  39145.220554   2087.771193      0.414911      3.088800  40.991536   
12   32110.642762   3509.044619      0.171196    341.304880  36.828260   
84   26484.861943  10374.965290      0.078372   3127.910319  40.165289   
87   22282.173535  14197.577076      0.419216   1768.808421  52.765957   
94   14149.854392   7581.491983      0.454893    104.426983  45.752928   
119  13580.574173   7195.178325      0.254607     47.593106  74.539632   
21   13164.275020   2856.300564      0.567771      0.000000  64.759725   
85   10882.694514   2176.833824      0.923372    256.342096  43.522802   
63    9066.088858   2500.336646      0.314640    441.137409  40.031898   
117   8430.527847   2361.424572      0.623784     55.940419  54.803866   
143   9747.273777   2599.761671      0